In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/football-match-probability-prediction/sample_submission.csv
/kaggle/input/football-match-probability-prediction/train_target_and_scores.csv
/kaggle/input/football-match-probability-prediction/train.csv
/kaggle/input/football-match-probability-prediction/test.csv


In [71]:
import torch
torch.cuda.is_available()

True

In [72]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier

In [73]:
df_train = pd.read_csv('/kaggle/input/football-match-probability-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/football-match-probability-prediction/test.csv')

/tmp/ipykernel_34/2064435966.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('/kaggle/input/football-match-probability-prediction/train.csv')


In [74]:
df_train.shape

(110938, 190)

In [75]:
def check_empty_values_percent(df):
    return (df.isnull().sum() / df.shape[0] * 100).sort_values()

In [76]:
check_empty_values_percent(df_train)

league_id                      0.000000
id                             0.000000
target                         0.000000
match_date                     0.000000
away_team_name                 0.000901
                                ...    
away_team_history_coach_8     22.732517
home_team_history_coach_9     23.195839
away_team_history_coach_9     23.571725
home_team_history_coach_10    24.025131
away_team_history_coach_10    24.454200
Length: 190, dtype: float64

In [77]:
check_empty_values_percent(df_test)

id                             0.000000
home_team_name                 0.000000
away_team_name                 0.000000
match_date                     0.000000
league_name                    0.000000
                                ...    
away_team_history_coach_8     28.507379
home_team_history_coach_9     28.875961
away_team_history_coach_9     28.877336
home_team_history_coach_10    29.239042
away_team_history_coach_10    29.278926
Length: 189, dtype: float64

In [78]:
def preprocess(df, name):
    df = df.drop(['id', 'home_team_coach_id', 'away_team_coach_id', 'home_team_name', 'away_team_name', 'league_name', 'league_id'], axis=1)
    for i in range(1, 11):
        df = df.drop([f'home_team_history_coach_{i}', f'home_team_history_league_id_{i}',
                              f'away_team_history_coach_{i}', f'away_team_history_league_id_{i}'], axis=1)
    
    for col in df.columns:
        if 'date' in col:
            df[col] = pd.to_datetime(df[col])
            
    df['is_cup'] = df['is_cup'].astype(pd.Float64Dtype())
    
    if name == 'train':
        le = LabelEncoder()
        df['target'] = le.fit_transform(df['target'])
    
    return df

In [79]:
df_train = preprocess(df_train, 'train')
df_test = preprocess(df_test, 'test')

In [80]:
check_empty_values_percent(df_train)

target                                   0.000000
match_date                               0.000000
is_cup                                   0.000901
home_team_history_match_date_1           1.044728
home_team_history_goal_1                 1.044728
                                          ...    
away_team_history_goal_10               12.814365
away_team_history_match_date_10         12.814365
away_team_history_rating_10             12.820675
away_team_history_opponent_rating_10    12.820675
away_team_history_is_cup_10             13.357912
Length: 143, dtype: float64

In [81]:
check_empty_values_percent(df_test)

match_date                              0.000000
is_cup                                  0.000000
away_team_history_match_date_1          0.671150
away_team_history_opponent_goal_1       0.671150
away_team_history_goal_1                0.671150
                                          ...   
away_team_history_is_play_home_10       8.675441
away_team_history_goal_10               8.675441
away_team_history_rating_10             8.689194
away_team_history_opponent_rating_10    8.689194
away_team_history_is_cup_10             8.828100
Length: 142, dtype: float64

In [82]:
corr = df_train.corr()['target']
print('maximum', abs(corr.sort_values(ascending=False).head(10)))
print('minimum', abs(corr.sort_values(ascending=False).tail(10)))

maximum target                                 1.000000
home_team_history_rating_2             0.108375
home_team_history_rating_1             0.104404
away_team_history_opponent_rating_1    0.104002
away_team_history_opponent_rating_2    0.101542
home_team_history_rating_3             0.098628
home_team_history_rating_5             0.096691
home_team_history_rating_6             0.095669
home_team_history_rating_4             0.095663
away_team_history_opponent_rating_4    0.094041
Name: target, dtype: float64
minimum home_team_history_opponent_rating_4    0.106171
away_team_history_rating_5             0.106624
away_team_history_rating_6             0.107669
away_team_history_rating_3             0.109831
away_team_history_rating_4             0.110319
home_team_history_opponent_rating_3    0.114486
home_team_history_opponent_rating_2    0.116356
away_team_history_rating_2             0.117092
away_team_history_rating_1             0.123964
home_team_history_opponent_rating_1    0.12

In [83]:
df_train

,target,match_date,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,...,away_team_history_opponent_rating_1,away_team_history_opponent_rating_2,away_team_history_opponent_rating_3,away_team_history_opponent_rating_4,away_team_history_opponent_rating_5,away_team_history_opponent_rating_6,away_team_history_opponent_rating_7,away_team_history_opponent_rating_8,away_team_history_opponent_rating_9,away_team_history_opponent_rating_10
0,0,2019-12-01 00:45:00,0.0,2019-11-26 00:10:00,2019-11-10 18:30:00,2019-11-03 18:30:00,2019-10-29 22:10:00,2019-10-19 20:45:00,2019-10-04 22:00:00,2019-09-28 23:00:00,...,5.253225,2.819962,3.114820,3.283603,2.574183,5.887331,4.570225,2.843268,3.057739,4.684664
1,2,2019-12-01 01:00:00,0.0,2019-11-27 21:00:00,2019-11-24 01:00:00,2019-11-10 01:00:00,2019-11-02 21:30:00,2019-10-27 01:00:00,2019-10-24 01:00:00,2019-10-21 00:00:00,...,6.881762,6.840558,8.700363,5.156150,3.864878,6.237028,6.871904,5.716200,7.960465,9.994456
2,1,2019-12-01 01:00:00,0.0,2019-11-28 01:15:00,2019-11-21 01:30:00,2019-11-03 21:30:00,2019-10-27 01:00:00,2019-10-20 22:00:00,2019-10-17 01:00:00,2019-10-05 21:00:00,...,5.487338,7.697712,4.186315,5.183071,6.176583,4.054209,4.650054,5.284983,5.299259,3.862791
3,0,2019-12-01 01:00:00,0.0,2019-11-28 01:00:00,2019-11-23 03:10:00,2019-11-10 03:00:00,2019-11-04 02:00:00,2019-11-01 03:00:00,2019-10-27 01:00:00,2019-10-20 17:00:00,...,5.998800,6.548433,4.576617,8.342183,4.878850,6.208425,7.483589,5.587387,7.301455,7.766762
4,2,2019-12-01 01:00:00,0.0,2019-11-27 18:00:00,2019-11-24 01:00:00,2019-11-10 02:00:00,2019-11-03 01:00:00,2019-10-27 20:30:00,2019-10-24 00:00:00,2019-10-20 18:00:00,...,5.575180,5.901842,5.751275,4.163600,8.559950,4.059211,8.935007,5.888077,4.413683,7.031962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110933,1,2021-04-30 23:00:00,0.0,2021-04-23 23:00:00,2021-04-18 23:00:00,2020-12-10 21:00:00,2020-12-09 00:00:00,2020-12-03 21:00:00,2020-11-30 00:00:00,2020-11-26 21:00:00,...,7.239339,9.332287,7.058742,6.961725,5.757350,7.712525,5.875017,6.233200,5.785250,3.224300
110934,0,2021-04-30 23:00:00,0.0,2021-04-23 23:30:00,2021-04-09 19:00:00,2021-04-03 00:00:00,2021-03-15 19:00:00,2021-03-11 19:00:00,2021-03-04 00:30:00,2021-02-25 00:30:00,...,7.421600,8.467391,6.183761,6.553486,15.376050,15.843563,4.510164,3.885800,12.142436,8.047688
110935,1,2021-04-30 23:00:00,0.0,2021-04-28 23:00:00,2021-04-25 01:00:00,2021-04-22 23:00:00,2021-04-21 01:00:00,2021-03-14 18:00:00,2021-03-10 18:00:00,2021-03-06 18:00:00,...,3.081836,3.070258,2.536957,5.304985,3.004767,4.952157,3.257969,4.472350,7.371638,6.301017
110936,0,2021-05-01 00:00:00,0.0,2021-04-24 00:00:00,2021-04-18 19:00:00,2021-04-12 19:00:00,2021-04-02 19:00:00,2021-03-27 19:00:00,2021-02-14 13:30:00,2021-02-11 21:30:00,...,5.888744,6.347747,5.557890,4.190300,6.666050,5.035318,6.378186,6.726760,5.217356,7.172523


In [84]:
def create_feature(df):
    df['rest_days_home'] = (df['match_date'] - df['home_team_history_match_date_1']).dt.days
    df['rest_days_away'] = (df['match_date'] - df['away_team_history_match_date_1']).dt.days
    
    df = df.drop(['match_date'], axis=1)
    for i in range(1, 11):
        df = df.drop([f'home_team_history_match_date_{i}', f'away_team_history_match_date_{i}'], axis=1)
        
    df['home_rating_mean'] = df[['home_team_history_rating_1', 'home_team_history_rating_2', 'home_team_history_rating_3', 'home_team_history_rating_4', 
                      'home_team_history_rating_5', 'home_team_history_rating_6', 'home_team_history_rating_7', 'home_team_history_rating_8', 
                      'home_team_history_rating_9', 'home_team_history_rating_10']].mean(axis=1)
    
    
    df['away_rating_mean'] = df[['away_team_history_rating_1', 'away_team_history_rating_2', 'away_team_history_rating_3', 'away_team_history_rating_4', 
                      'away_team_history_rating_5', 'away_team_history_rating_6', 'away_team_history_rating_7', 'away_team_history_rating_8', 
                      'away_team_history_rating_9', 'away_team_history_rating_10']].mean(axis=1)
    
    df['home_opponent_rating_mean'] = df[['home_team_history_opponent_rating_1', 'home_team_history_opponent_rating_2', 'home_team_history_opponent_rating_3', 
                          'home_team_history_opponent_rating_4', 'home_team_history_opponent_rating_5', 'home_team_history_opponent_rating_6', 
                          'home_team_history_opponent_rating_7', 'home_team_history_opponent_rating_8', 'home_team_history_opponent_rating_9', 
                          'home_team_history_opponent_rating_10']].mean(axis=1)
    
    df['away_opponent_rating_mean'] = df[['away_team_history_opponent_rating_1', 'away_team_history_opponent_rating_2', 'away_team_history_opponent_rating_3', 
                        'away_team_history_opponent_rating_4', 'away_team_history_opponent_rating_5', 'away_team_history_opponent_rating_6', 
                        'away_team_history_opponent_rating_7', 'away_team_history_opponent_rating_8', 'away_team_history_opponent_rating_9', 
                        'away_team_history_opponent_rating_10']].mean(axis=1)
    
    df['home_goals'] = df[['home_team_history_goal_1', 'home_team_history_goal_2', 'home_team_history_goal_3', 'home_team_history_goal_4',
                     'home_team_history_goal_5', 'home_team_history_goal_6', 'home_team_history_goal_7', 'home_team_history_goal_8',
                     'home_team_history_goal_9', 'home_team_history_goal_10']].mean(axis=1)
    
    df['away_goals'] = df[['away_team_history_goal_1', 'away_team_history_goal_2', 'away_team_history_goal_3', 'away_team_history_goal_4', 
                    'away_team_history_goal_5', 'away_team_history_goal_6', 'away_team_history_goal_7', 'away_team_history_goal_8', 
                    'away_team_history_goal_9', 'away_team_history_goal_10']].mean(axis=1)
    
    df['home_opponent_goals'] = df[['home_team_history_opponent_goal_1', 'home_team_history_opponent_goal_2', 'home_team_history_opponent_goal_3', 
                     'home_team_history_opponent_goal_4', 'home_team_history_opponent_goal_5', 'home_team_history_opponent_goal_6', 
                     'home_team_history_opponent_goal_7', 'home_team_history_opponent_goal_8', 'home_team_history_opponent_goal_9', 
                     'home_team_history_opponent_goal_10']].mean(axis=1)
    
    df['away_opponent_goals'] = df[['away_team_history_opponent_goal_1', 'away_team_history_opponent_goal_2', 'away_team_history_opponent_goal_3', 
                     'away_team_history_opponent_goal_4', 'away_team_history_opponent_goal_5', 'away_team_history_opponent_goal_6', 
                     'away_team_history_opponent_goal_7', 'away_team_history_opponent_goal_8', 'away_team_history_opponent_goal_9',
                     'away_team_history_opponent_goal_10']].mean(axis=1)
    
    df['home_iscup'] = df[['home_team_history_is_cup_1', 'home_team_history_is_cup_2', 'home_team_history_is_cup_3', 'home_team_history_is_cup_4', 
             'home_team_history_is_cup_5', 'home_team_history_is_cup_6', 'home_team_history_is_cup_7', 'home_team_history_is_cup_8', 
             'home_team_history_is_cup_9', 'home_team_history_is_cup_10']].mean(axis=1)

    df['away_iscup'] = df[['away_team_history_is_cup_1', 'away_team_history_is_cup_2', 'away_team_history_is_cup_3', 'away_team_history_is_cup_4', 
                 'away_team_history_is_cup_5', 'away_team_history_is_cup_6', 'away_team_history_is_cup_7', 'away_team_history_is_cup_8', 
                 'away_team_history_is_cup_9', 'away_team_history_is_cup_10']].mean(axis=1)

    df['home_ishome'] = df[['home_team_history_is_play_home_1', 'home_team_history_is_play_home_2', 'home_team_history_is_play_home_3', 'home_team_history_is_play_home_4', 
                  'home_team_history_is_play_home_5', 'home_team_history_is_play_home_6', 'home_team_history_is_play_home_7', 'home_team_history_is_play_home_8', 
                  'home_team_history_is_play_home_9', 'home_team_history_is_play_home_10']].mean(axis=1)

    df['away_ishome'] = df[['away_team_history_is_play_home_1', 'away_team_history_is_play_home_2', 'away_team_history_is_play_home_3', 'away_team_history_is_play_home_4', 
                  'away_team_history_is_play_home_5', 'away_team_history_is_play_home_6', 'away_team_history_is_play_home_7', 'away_team_history_is_play_home_8',
                  'away_team_history_is_play_home_9', 'away_team_history_is_play_home_10']].mean(axis=1)
    
    return df

In [85]:
df_train = create_feature(df_train)
df_test = create_feature(df_test)

In [87]:
corr = df_train.corr()['target']
print('maximum', corr.sort_values(ascending=False).head(10))
print('minimum', corr.sort_values(ascending=False).tail(10))

maximum target                                 1.000000
home_rating_mean                       0.138057
away_opponent_rating_mean              0.134835
away_opponent_goals                    0.133439
home_goals                             0.130847
home_team_history_rating_2             0.108375
home_team_history_rating_1             0.104404
away_team_history_opponent_rating_1    0.104002
away_team_history_opponent_rating_2    0.101542
home_team_history_rating_3             0.098628
Name: target, dtype: float64
minimum away_team_history_rating_4            -0.110319
home_team_history_opponent_rating_3   -0.114486
home_team_history_opponent_rating_2   -0.116356
away_team_history_rating_2            -0.117092
away_team_history_rating_1            -0.123964
home_team_history_opponent_rating_1   -0.124314
away_goals                            -0.141275
home_opponent_goals                   -0.149250
away_rating_mean                      -0.157793
home_opponent_rating_mean             -0.15

In [88]:
df_train = df_train.fillna(df_train.mean())
df_test = df_test.fillna(df_test.mean())

In [89]:
df_train

,target,is_cup,home_team_history_is_play_home_1,home_team_history_is_play_home_2,home_team_history_is_play_home_3,home_team_history_is_play_home_4,home_team_history_is_play_home_5,home_team_history_is_play_home_6,home_team_history_is_play_home_7,home_team_history_is_play_home_8,...,home_opponent_rating_mean,away_opponent_rating_mean,home_goals,away_goals,home_opponent_goals,away_opponent_goals,home_iscup,away_iscup,home_ishome,away_ishome
0,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,4.400204,3.808902,1.3,1.6,1.0,0.9,0.0,0.5,0.5,0.6
1,2,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,4.546772,6.822376,0.9,0.8,1.0,1.4,0.0,0.0,0.5,0.5
2,1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,5.547477,5.188232,1.9,2.1,1.7,1.1,0.0,0.0,0.5,0.5
3,0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,4.983015,6.469250,1.8,2.4,1.3,2.0,0.0,0.1,0.5,0.5
4,2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,5.192975,6.027979,1.8,1.8,1.4,1.0,0.0,0.0,0.5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110933,1,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,4.686841,6.517974,1.7,1.7,0.9,1.4,0.0,0.0,0.5,0.5
110934,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,6.962505,8.843194,2.6,1.3,1.4,1.7,0.2,0.0,0.5,0.5
110935,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,4.690324,4.335393,1.0,1.2,1.1,0.2,0.2,0.2,0.4,0.6
110936,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,5.605793,5.918087,1.0,0.7,1.4,0.7,0.0,0.0,0.5,0.6


In [90]:
y = df_train['target']
X = df_train.drop(['target'], axis=1)

In [91]:
le = LabelEncoder()
y = le.fit_transform(y)

In [97]:
X.shape, y.shape

((110938, 135), (110938,))

In [96]:
model = CatBoostClassifier(verbose=False, learning_rate=0.1,iterations = 1000, task_type="GPU")
model.fit(X, y)

In [38]:
predCat = model.predict_proba(df_test)

In [39]:
ss2 = pd.read_csv('/kaggle/input/football-match-probability-prediction/sample_submission.csv')
ss2

,id,home,draw,away
0,17761448,0.333,0.333,0.333
1,17695487,0.333,0.333,0.333
2,17715496,0.333,0.333,0.333
3,17715493,0.333,0.333,0.333
4,17715492,0.333,0.333,0.333
...,...,...,...,...
72706,18450246,0.333,0.333,0.333
72707,18164889,0.333,0.333,0.333
72708,18449018,0.333,0.333,0.333
72709,17958831,0.333,0.333,0.333


In [40]:
ss2[['home', 'draw', 'away']] = predCat

Kaagle score: 1.23253